Step 1) Load the precinct Data

In [46]:
import pandas as pd
from shapely.geometry import Polygon,shape,MultiPolygon
import shapefile as shp
import ast
from collections import deque
import random


In [ ]:
# load in data, and get the min and max district population count for each NJ district
NUM_DISTRICTS=12

nj_precinct_data = pd.read_csv('precinct-data-congress-nj.csv')
total_population=int(nj_precinct_data["Total_2020_Total"].sum())
district_population=int(total_population/NUM_DISTRICTS)

min_pop, max_pop = int(district_population * .95), int(district_population * 1.05)

shpfile = 'nj_vtd_2020_bound.shp'
dbffile = 'nj_vtd_2020_bound.dbf'
shxfile = 'nj_vtd_2020_bound.shx'
prjfile = 'nj_vtd_2020_bound.prj'
cpgfile = 'nj_vtd_2020_bound.cpg'

sf = shp.Reader(shp=shpfile, shx=shxfile, dbf=dbffile, prj=prjfile, cpg=cpgfile)
nj_precinct_boundaries = {}
for sr in sf.iterShapeRecords():
    geom = sr.shape
    rec = sr.record
    nj_precinct_boundaries[rec[3]] = geom

nj_contiguity_df = pd.read_csv('Contiguity_nj.csv', header=None)
nj_contiguity_df.columns = ['Precinct', 'Neighbors']
nj_contiguity = {}
for index, row in nj_contiguity_df.iterrows():
    neighbors = ast.literal_eval(row['Neighbors'])
    nj_contiguity[row['Precinct']] = [str(n) for n in neighbors] if isinstance(neighbors[0], int) else neighbors
total_population

9288994

In [48]:
# setup floodfill

floodfill_map=nj_precinct_data.copy()
floodfill_map['District']=0
seed_precincts = {}

for index, row in floodfill_map.iterrows():
    geoid = row['GEOID20']
    s = shape(nj_precinct_boundaries[geoid])
    centroid = s.centroid
   
    district_num = 0
    if centroid.x <= -74.72:
        if centroid.y <= 39.3: district_num = 1
        elif centroid.y <= 39.7: district_num = 3
        elif centroid.y <= 40.1: district_num = 5
        elif centroid.y <= 40.5: district_num = 7
        elif centroid.y <= 40.9: district_num = 9
        else: district_num = 11
    else:
        if centroid.y <= 39.3: district_num = 2
        elif centroid.y <= 39.7: district_num = 4
        elif centroid.y <= 40.1: district_num = 6
        elif centroid.y <= 40.5: district_num = 8
        elif centroid.y <= 40.9: district_num = 10
        else: district_num = 12

    if district_num != 0 and district_num not in seed_precincts:
        seed_precincts[district_num] = geoid

    if len(seed_precincts) == NUM_DISTRICTS:
        break



In [50]:
# implement floodfill
district_current_pop = {i: 0 for i in range(1, NUM_DISTRICTS + 1)}
district_queues = {i: deque() for i in range(1, NUM_DISTRICTS + 1)}

for district_num, geoid in seed_precincts.items():
    floodfill_map.loc[floodfill_map['GEOID20'] == geoid, 'District'] = district_num
    pop = floodfill_map.loc[floodfill_map['GEOID20'] == geoid, 'Total_2020_Total'].iloc[0]
    district_current_pop[district_num] += pop
    district_queues[district_num].append(geoid)

assigned_precincts = set(seed_precincts.values())
unassigned_count = len(floodfill_map) - len(assigned_precincts)

# flood fill
while unassigned_count > 0:
    growable_districts = [
        d for d in range(1, NUM_DISTRICTS + 1)
        if district_queues[d] and district_current_pop[d] < max_pop
    ]
    if not growable_districts:
        print("Cannot grow further, but still unassigned precincts left.")
        break

    # sort by population, get district with smallest population, get frontier precinct, and get unassigned neighbors
    growable_districts.sort(key=lambda d: district_current_pop[d])
    selected_district = growable_districts[0]
    frontier_precinct = district_queues[selected_district].popleft()
    unassigned_neighbors = [
        neighbor for neighbor in nj_contiguity.get(frontier_precinct, [])
        if floodfill_map.loc[floodfill_map['GEOID20'] == neighbor, 'District'].iloc[0] == 0
    ]

    if not unassigned_neighbors:
        continue

    # get neighbors that wont exceed max population
    possible_neighbors = [
        n for n in unassigned_neighbors
        if (floodfill_map.loc[floodfill_map['GEOID20'] == n, 'Total_2020_Total'].iloc[0] +
            district_current_pop[selected_district]) <= max_pop
    ]

    if not possible_neighbors:
        still_has_unassigned = any(
            floodfill_map.loc[floodfill_map['GEOID20'] == n, 'District'].iloc[0] == 0
            for n in nj_contiguity.get(frontier_precinct, [])
        )
        if still_has_unassigned:
            district_queues[selected_district].append(frontier_precinct)
        continue

    next_precinct = random.choice(possible_neighbors)
    floodfill_map.loc[floodfill_map['GEOID20'] == next_precinct, 'District'] = selected_district
    pop_add = floodfill_map.loc[floodfill_map['GEOID20'] == next_precinct, 'Total_2020_Total'].iloc[0]
    district_current_pop[selected_district] += pop_add
    district_queues[selected_district].append(next_precinct)
    unassigned_count -= 1
    still_has_unassigned = any(
        floodfill_map.loc[floodfill_map['GEOID20'] == n, 'District'].iloc[0] == 0
        for n in nj_contiguity.get(frontier_precinct, [])
    )

    if still_has_unassigned:
        district_queues[selected_district].append(frontier_precinct)

print("District populations after flood fill:")
for d in range(1, NUM_DISTRICTS + 1):
    print(f"District {d}: {district_current_pop[d]} (min: {min_pop}, max: {max_pop})")

floodfill_map[['GEOID20', 'District']].to_csv('fair_nj_map.csv', index=False)

Cannot grow further, but still unassigned precincts left.
District populations after flood fill:
District 1: 365 (min: 735377, max: 812786)
District 2: 550 (min: 735377, max: 812786)
District 3: 2184 (min: 735377, max: 812786)
District 4: 1240 (min: 735377, max: 812786)
District 5: 877 (min: 735377, max: 812786)
District 6: 1544 (min: 735377, max: 812786)
District 7: 3196 (min: 735377, max: 812786)
District 8: 601 (min: 735377, max: 812786)
District 9: 1098 (min: 735377, max: 812786)
District 10: 1366 (min: 735377, max: 812786)
District 11: 595 (min: 735377, max: 812786)
District 12: 2088 (min: 735377, max: 812786)
